In [1]:
from environments import ZeroDCleanupEnv
from agents import QAgent
import numpy as np
from collections import defaultdict
import torch
import sys
from tqdm import tqdm

2023-11-22 00:19:52,115	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
np.set_printoptions(threshold=sys.maxsize)

cpu


In [3]:
num_agents = 10
agent_ids = [str(i) for i in range(num_agents)]
reward_multiplier = 10

In [4]:
pp = False
verbose = False

In [5]:
num_episodes = 200
verbose_episode = num_episodes - 1
max_steps_per_episode = 1000
state_dim = 4
action_dim = 2

In [6]:
thresholdDepletion = 0.4
thresholdRestoration = 0.0
wasteSpawnProbability = 0.5
appleRespawnProbability = 0.05
dirt_multiplier = 10

area = 150

In [7]:
lr = 0.01
gamma = 0.99
epsilon = 0.5
epsilon_decay = 0.9999
epsilon_min = 0.05
batch_size = 128

In [8]:
env = ZeroDCleanupEnv(agent_ids, 
                      num_agents=num_agents, 
                      area=area, 
                      thresholdDepletion=thresholdDepletion,
                      thresholdRestoration=thresholdRestoration,
                      wasteSpawnProbability=wasteSpawnProbability,
                      appleRespawnProbability=appleRespawnProbability,
                      dirt_multiplier=dirt_multiplier)

agents = {
    agent_id: QAgent(device=device,
                    num_action_outputs=1,
                    action_size=action_dim,
                    state_dim=state_dim,
                    lr=lr,
                    gamma=gamma,
                    epsilon=epsilon,
                    epsilon_decay=epsilon_decay,
                    epsilon_min=epsilon_min,
                    batch_size=batch_size,
                    q_layers=[
                        (state_dim, 16),
                        (16, 8),
                        #(64, 64),
                        #(16, 16),
                        (8, action_dim)
                    ],
                    verbose=False)
    for agent_id in agent_ids
}

In [9]:
ending_ep_rewards = []
avg_agent_rewards = []
agent_reward_variances = []
weight_graph = defaultdict(list)

In [10]:
import uuid
import os

if not os.path.exists("results"):
    os.mkdir("results")

test_id = uuid.uuid4()
os.mkdir("results/q_simple_" + str(test_id))
f = open("results/q_simple_" + str(test_id) + "/logs.txt", "x")

In [11]:
for episode in range(num_episodes):
    print(f"========= Episode {episode} =========")
    f.write(f"========= Episode {episode} =========\n")

    states, info = env.reset()

    print(f"info: {info}")
    f.write(f"info: {info}\n")

    for step in tqdm(range(max_steps_per_episode)):
        actions = {agent_id: agents[agent_id].act(states[agent_id]) for agent_id in agent_ids}
        next_states, rewards, dones, _, info = env.step(actions)
        
        for agent_id in agent_ids:
            agents[agent_id].step(states[agent_id], actions[agent_id], rewards[agent_id], next_states[agent_id])

        if episode > verbose_episode:
            print(f"========= Step {step} =========")
            print(f"info: {info}")
            f.write(f"========= Step {step} =========\n")
            f.write(f"info: {info}\n")

        states = next_states

        if dones["__all__"]:
            break 

    ending_reward = info["total_apple_consumed"]
    total_reward_by_agent = info["total_reward_by_agent"]

    print(f"ending reward: {ending_reward}")
    #print(f"reward graph: {reward_graph}")
    print(f"========= End of Episode {episode} =========")
    
    f.write(f"ending reward: {ending_reward}\n")
    #f.write(f"reward graph: {reward_graph}\n")
    f.write(f"========= End of Episode {episode} =========\n")

    ending_ep_rewards.append(ending_reward)
    avg_agent_rewards.append(ending_reward / num_agents)
    agent_reward_variances.append(np.var(list(total_reward_by_agent.values())))
        

========= Episode 0 =========
info: {'total_apple_consumed': 0, 'step_apple_consumed': 0, 'apple': 0, 'dirt': 78, 'picker': 0, 'cleaner': 0, 'total_reward_by_agent': {'7': 0, '1': 0, '8': 0, '6': 0, '5': 0, '0': 0, '4': 0, '2': 0, '3': 0, '9': 0}}


 13%|█▎        | 129/1000 [00:00<00:00, 939.51it/s]

100%|█████████▉| 999/1000 [00:08<00:00, 112.88it/s]


ending reward: 2614
========= End of Episode 0 =========
========= Episode 1 =========
info: {'total_apple_consumed': 0, 'step_apple_consumed': 0, 'apple': 0, 'dirt': 78, 'picker': 0, 'cleaner': 0, 'total_reward_by_agent': {'7': 0, '1': 0, '8': 0, '6': 0, '5': 0, '0': 0, '4': 0, '2': 0, '3': 0, '9': 0}}


100%|█████████▉| 999/1000 [00:09<00:00, 101.49it/s]


...

100%|█████████▉| 999/1000 [00:09<00:00, 104.68it/s]


ending reward: 74
========= End of Episode 176 =========
========= Episode 177 =========
info: {'total_apple_consumed': 0, 'step_apple_consumed': 0, 'apple': 0, 'dirt': 78, 'picker': 0, 'cleaner': 0, 'total_reward_by_agent': {'7': 0, '1': 0, '8': 0, '6': 0, '5': 0, '0': 0, '4': 0, '2': 0, '3': 0, '9': 0}}


100%|█████████▉| 999/1000 [00:09<00:00, 107.10it/s]


ending reward: 36
========= End of Episode 177 =========
========= Episode 178 =========
info: {'total_apple_consumed': 0, 'step_apple_consumed': 0, 'apple': 0, 'dirt': 78, 'picker': 0, 'cleaner': 0, 'total_reward_by_agent': {'7': 0, '1': 0, '8': 0, '6': 0, '5': 0, '0': 0, '4': 0, '2': 0, '3': 0, '9': 0}}


 89%|████████▊ | 886/1000 [00:08<00:00, 115.98it/s]

In [ ]:
test_stats = []
num_test_episodes = 15
print("========= Testing =========")
f.write("========= Testing =========\n")

for episode in range(num_test_episodes):
    test_stats.append({
        "num_apples": [],
        "num_dirt": [],
        "pickers": [],
        "cleaners": [],
        "total_reward": 0,
        "avg_agent_rewards": 0,
        "agent_reward_variances": 0,
        "total_reward_by_agent": [],
    })

    states, info = env.reset()
    test_stats[-1]["num_apples"].append(info["apple"])
    test_stats[-1]["num_dirt"].append(info["dirt"])
    test_stats[-1]["pickers"].append(info["picker"])
    test_stats[-1]["cleaners"].append(info["cleaner"])

    print(f"info: {info}")
    f.write(f"info: {info}\n")

    for step in tqdm(range(max_steps_per_episode)):
        actions = {agent_id: agents[agent_id].act(states[agent_id]) for agent_id in agent_ids}
        next_states, rewards, dones, _, info = env.step(actions)
        test_stats[-1]["num_apples"].append(info["apple"])
        test_stats[-1]["num_dirt"].append(info["dirt"])
        test_stats[-1]["pickers"].append(info["picker"])
        test_stats[-1]["cleaners"].append(info["cleaner"])

        if episode > verbose_episode:
            print(f"========= Step {step} =========")
            print(f"info: {info}")
            f.write(f"========= Step {step} =========\n")
            f.write(f"info: {info}\n")

        states = next_states

        if dones["__all__"]:
            break 

    print(f"ending info: {info}")

    test_reward = info["total_apple_consumed"]

    test_stats[-1]["total_reward"] = test_reward
    test_stats[-1]["avg_agent_rewards"] = test_reward / num_agents
    test_stats[-1]["agent_reward_variances"] = np.var(list(info["total_reward_by_agent"].values()))
    test_stats[-1]["total_reward_by_agent"] = list(info["total_reward_by_agent"].values())

    print('total reward:', test_reward)
    print('avg agent reward:', test_reward / num_agents)
    print('reward by agent:', info["total_reward_by_agent"])
    print('agent reward variances:', np.var(list(info["total_reward_by_agent"].values())))
    #print(f"reward graph: {reward_graph}")
    print(f"========= End of Test =========")

    f.write(f"ending info: {info}\n")
    #f.write(f"reward graph: {reward_graph}\n")
    f.write(f"========= End of Test =========\n")



In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.title("Reward by Training Episode")
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.plot(ending_ep_rewards)
plt.show()

In [ ]:
plt.figure()
plt.title("Reward by Test Episode")
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.plot([stat['total_reward'] for stat in test_stats])
plt.show()

In [ ]:
plt.figure()
plt.title("Average Number of Apples and Dirt by Test Step")
plt.xlabel("Step")
plt.ylabel("Number")
avg_num_apples = []
avg_num_dirt = []
for i in range(max_steps_per_episode):
    avg_num_apples.append(np.mean([stat['num_apples'][i] for stat in test_stats]))
    avg_num_dirt.append(np.mean([stat['num_dirt'][i] for stat in test_stats]))
plt.plot(avg_num_apples, label="Apples")
plt.plot(avg_num_dirt, label="Dirt")
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.title("Average Number of Pickers and Cleaners by Test Step")
plt.xlabel("Step")
plt.ylabel("Number")
avg_num_pickers = []
avg_num_cleaners = []
for i in range(max_steps_per_episode):
    avg_num_pickers.append(np.mean([stat['pickers'][i] for stat in test_stats]))
    avg_num_cleaners.append(np.mean([stat['cleaners'][i] for stat in test_stats]))
plt.plot(avg_num_pickers, label="Pickers")
plt.plot(avg_num_cleaners, label="Cleaners")
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.title("Average Ratios by Test Step")
plt.xlabel("Step")
plt.ylabel("Ratio")
avg_apple_dirt_ratios = []
apple_dirt_steps = []
avg_picker_cleaner_ratios = []
picker_cleaner_steps = []
for i in range(max_steps_per_episode):
    #avg_apple_dirt_ratios.append(np.mean([stat['num_apples'][i] / (stat['num_dirt'][i] + 1e-8) for stat in test_stats]))
    #avg_picker_cleaner_ratios.append(np.mean([stat['pickers'][i] / (stat['cleaners'][i]) for stat in test_stats if stat['cleaners'][i] > 0]))
    apple_dirt = []
    picker_cleaner = []
    for stat in test_stats:
        if stat['num_dirt'][i] > 0:
            apple_dirt.append(stat['num_apples'][i] / stat['num_dirt'][i])
        if stat['cleaners'][i] > 0:
            picker_cleaner.append(stat['pickers'][i] / stat['cleaners'][i])
    if len(apple_dirt) > 0:
        avg_apple_dirt_ratios.append(np.mean(apple_dirt))
        apple_dirt_steps.append(i)
    if len(picker_cleaner) > 0:
        avg_picker_cleaner_ratios.append(np.mean(picker_cleaner))
        picker_cleaner_steps.append(i)

plt.plot(apple_dirt_steps, avg_apple_dirt_ratios, label="Apples/Dirt")
plt.plot(picker_cleaner_steps, avg_picker_cleaner_ratios, label="Pickers/Cleaners")
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.title("Reward Variance by Training Episode")
plt.xlabel("Episode")
plt.ylabel("Variance")
plt.plot(agent_reward_variances)
plt.show()

In [ ]:
plt.figure()
plt.title("Reward Standard Deviation by Test Episode")
plt.xlabel("Episode")
plt.ylabel("Variance")
plt.plot([np.sqrt(stat['agent_reward_variances']) for stat in test_stats])
plt.show()

In [ ]:
plt.figure()
plt.title("Average Reward Curve")
plt.xlabel("Reward")
plt.ylabel("Frequency")
reward_curve = [0.] * num_agents
for stat in test_stats:
    reward_by_agent = sorted(stat['total_reward_by_agent'])
    for i in range(num_agents):
        reward_curve[i] += reward_by_agent[i]
for i in range(num_agents):
    reward_curve[i] /= num_test_episodes

plt.hist(reward_curve, bins=num_agents)
plt.show()